In [1]:
import os
import sys
sys.path.insert(0, os.path.relpath('../../../../..')) 

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)

In [2]:
court_data = pd.concat([pd.read_csv(f'../state/VA/VA_jail_data/circuit_criminal_{year}_anon_00.csv') for year in range(2010,2020)])
                      
court_data = court_data.drop(
    ['HearingJury', 'HearingPlea', 'HearingRoom', 'fips', 'Locality', 'DrivingRestrictions', 
     'RestrictionEffectiveDate', 'RestrictionEndDate', 'RestitutionPaid', 
     'RestitutionAmount', 'Military', 'TrafficFatality', 'person_id', 'Sex', 'Race', 'Address', 'Costs', 
     'FinesCostPaid', 'ProgramType', 'CourtDMVSurrender', 'DriverImprovementClinic', 'VAAlcoholSafetyAction', 
     'OffenseDate', 'ConcurrentConsecutive', 'LifeDeath', 'OperatorLicenseSuspensionTime', 'FineAmount', 'ArrestDate',
     'AmendedCharge', 'AmendedCodeSection', 'AmendedChargeType', 'ConcludedBy', 'DispositionDate'], axis=1)
court_data = court_data[court_data.DispositionCode == 'Guilty'].drop('DispositionCode', axis=1)
print(f"number of records: {len(court_data)}")
court_data.head()

/Users/pacopoler/.local/share/virtualenvs/pulse-data-Jd9N3JcA/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (39) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


number of records: 791960


,HearingDate,HearingResult,HearingType,Filed,Commencedby,Charge,CodeSection,ChargeType,Class,JailPenitentiary,SentenceTime,SentenceSuspended,ProbationType,ProbationTime,ProbationStarts,AppealedDate
10,2010-12-17,Appeal Withdrawn,Motion - Other Pre-Trial,2010-12-01,General District Court Appeal,PETIT LARC <$200 NOT FRM PERSN,18.2-96,Misdemeanor,1,Jail,180.0,170.0,NaN,NaN,NaN,NaN
11,2010-12-16,Sent,Trial,2010-05-11,General District Court Appeal,ELUDE POLICE - MISD.,A.46.2-817,Misdemeanor,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,2010-12-14,Appeal Withdrawn,Motion - Other Pre-Trial,2010-11-04,General District Court Appeal,100/55 RD,A.46.2-862,Misdemeanor,1,Jail,10.0,6.0,NaN,NaN,NaN,NaN
14,2010-12-09,Sent,Sentencing,2010-04-01,Direct Indictment,AGGRAVATED SEXUAL BATTERY,18.2-67.3,Felony,NaN,Penitentiary,7300.0,7300.0,Indefinite Supervision,NaN,NaN,NaN
15,2010-12-09,Sent,Sentencing,2010-04-01,Direct Indictment,ANIMATE OBJECT SEXUAL PENETRA,18.2-67.2,Felony,NaN,Penitentiary,14600.0,5475.0,Indefinite Supervision,NaN,NaN,NaN


In [3]:
~court_data.Charge.str.contains('FIREARM').apply(bool)

10        True
11        True
13        True
14        True
15        True
          ... 
224252    True
224253    True
224254    True
224256    True
224257    True
Name: Charge, Length: 791960, dtype: bool

In [4]:
drug_crimes = court_data[court_data.Charge.str.contains('SCH') 
                         | (court_data.Charge.str.contains('POSS') & ~court_data.Charge.str.contains('FIREARM').apply(bool)) 
                         | court_data.Charge.str.contains('MARIJ') 
                         | court_data.Charge.str.contains('DRUG')]
drug_crimes['time_step'] = drug_crimes.HearingDate.apply(lambda x: 12 * (int(x[:4]) - 2010) + int(x[5:7]))
drug_crimes.groupby(['ChargeType','Class']).agg({'SentenceTime': 'mean', 'HearingType': 'count'})

/Users/pacopoler/.local/share/virtualenvs/pulse-data-Jd9N3JcA/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


SentenceTime  HearingType
ChargeType  Class                           
Felony      1        435.178197          491
            2       2506.694915           64
            3       1956.034483           30
            4       1682.264000          405
            5       1444.859172        72364
            6       1337.884154        19333
            O       3650.000000            1
            U       2923.164874        30028
Infraction  1               NaN            1
            6               NaN            1
            U               NaN          199
Misdemeanor 1        272.372651         8097
            2        181.941176          981
            3        279.500000           75
            4        185.625000          110
            5               NaN            4
            6               NaN            2
            U         70.519632         5686

In [5]:
possession_crimes = drug_crimes[drug_crimes.CodeSection.str.contains('18.2-250').apply(bool)]
a = possession_crimes[(possession_crimes.ChargeType == 'Felony') & (possession_crimes.Class == '5') & (possession_crimes.Charge.str.contains('BUT'))]
a

,HearingDate,HearingResult,HearingType,Filed,Commencedby,Charge,CodeSection,ChargeType,Class,JailPenitentiary,SentenceTime,SentenceSuspended,ProbationType,ProbationTime,ProbationStarts,AppealedDate,time_step
97165,2011-11-09,Sent,Pre-Sentence Report,2011-06-23,Indictment,POSSESS GAMA-BUTYROLACTONEL,18.2-250,Felony,5,Penitentiary,730.0,730.0,Supervised,1095.0,NaN,NaN,23
116272,2012-08-20,Sent,Motion - Other Pre-Trial,2012-01-24,Indictment,POSSESS GAMA-BUTYROLACTONEL,18.2-250,Felony,5,Penitentiary,1825.0,1825.0,Supervised,1095.0,Release,NaN,32
119305,2012-01-06,Sent,Plea,2011-11-14,Indictment,POSSESS GAMA-BUTYROLACTONEL,18.2-250,Felony,5,Penitentiary,1825.0,1825.0,Supervised,365.0,Release,NaN,25
152401,2012-07-23,Sent,Deferred Disposition,2010-11-24,Indictment,POSSESS GAMA-BUTYROLACTONEL,18.2-250,Felony,5,Penitentiary,1825.0,1825.0,NaN,NaN,NaN,NaN,31
1666,2013-12-16,Granted,Hearing,1995-12-18,Indictment,POSSESS GAMA-BUTYROLACTONEL,18.2-250,Felony,5,Penitentiary,1460.0,1460.0,Supervised,1095.0,Sentencing,NaN,48
123797,2013-11-07,Sent,Trial,2004-10-08,Indictment,POSSESS GAMA-BUTYROLACTONEL,18.2-250,Felony,5,Penitentiary,1825.0,1825.0,NaN,NaN,NaN,NaN,47
34066,2015-09-04,Denied,Motion - Other Post-Trial,2002-07-15,Indictment,POSSESS GAMA-BUTYROLACTONEL,18.2-250,Felony,5,Penitentiary,1095.0,1095.0,Supervised,730.0,Sentencing,NaN,69
62002,2016-11-28,Sent,Plea,2016-09-20,Indictment,POSSESS GAMA-BUTYROLACTONEL,18.2-250,Felony,5,Jail,365.0,270.0,Unsupervised,365.0,NaN,NaN,83
26407,2017-09-22,Granted,Restricted O.L.,2017-05-24,Waiver,POSSESS GAMA-BUTYROLACTONEL,18.2-250,Felony,5,Time Served,730.0,730.0,Supervised,730.0,Sentencing,NaN,93
26845,2017-08-25,Granted,Restricted O.L.,2017-05-19,Waiver,POSSESS GAMA-BUTYROLACTONEL,18.2-250,Felony,5,Penitentiary,730.0,730.0,Supervised,365.0,Release,NaN,92


In [6]:
drug_crimes[drug_crimes.CodeSection.str.contains('18.2-250').apply(bool)].groupby(['ChargeType', 'Class'], dropna=False).agg({'SentenceTime': 'mean', 'HearingType': 'count'})

SentenceTime  HearingType
ChargeType  Class                           
Felony      1        614.603175           66
            2        637.500000            2
            3       1581.666667            3
            4       1095.000000            2
            5       1393.120518        49365
            6       1239.852941           38
            U       1466.777975          585
            NaN     1432.452253         3073
Misdemeanor 1        233.706945         2916
            2        180.826446          846
            3         90.000000           13
            4        197.333333           82
            5               NaN            1
            U         66.975188         5114
            NaN      122.569593         1950

# preprocessing

In [7]:
diversion_fractions = dict()
MAX_SENTENCE = 40 * 365

In [8]:
sch1_2_offenses = drug_crimes[(drug_crimes.CodeSection.str.contains('18.2-250').apply(bool)) 
                              & (drug_crimes.ChargeType == 'Felony')
                              & (drug_crimes.Class == '5')
                              & (~drug_crimes.Charge.str.contains('GAMA'))
                              & (~drug_crimes.Charge.str.contains('MARIJUANA'))]
sch1_2_offenses

,HearingDate,HearingResult,HearingType,Filed,Commencedby,Charge,CodeSection,ChargeType,Class,JailPenitentiary,SentenceTime,SentenceSuspended,ProbationType,ProbationTime,ProbationStarts,AppealedDate,time_step
55,2010-11-18,Sent,Plea,2010-06-03,Indictment,DRUGS: POSSESS SCH I OR II,18.2-250,Felony,5,Penitentiary,730.0,665.0,Indefinite Supervision,NaN,NaN,NaN,11
76,2010-11-04,Sent,Trial,2010-07-29,Indictment,POSS.OF CONTROLLED SUBSTANCE,18.2-250,Felony,5,Penitentiary,730.0,730.0,Indefinite Supervision,36135.0,NaN,NaN,11
135,2010-10-07,NaN,Motion - Other Post-Trial,2008-08-26,Indictment,DRUGS: POSSESS SCH I OR II,18.2-250,Felony,5,Penitentiary,730.0,730.0,Indefinite Supervision,NaN,NaN,NaN,10
171,2010-09-09,Sent,Sentencing,2010-03-03,Indictment,DRUGS: POSSESS SCH I OR II,18.2-250,Felony,5,Penitentiary,1825.0,1825.0,Indefinite Supervision,NaN,NaN,NaN,9
459,2010-05-06,Sent,Plea,2010-03-01,Indictment,DRUGS: POSSESS SCH I OR II,18.2-250,Felony,5,Jail,365.0,300.0,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223815,2019-03-27,Sentenced,Pre-Sentence Report,2018-09-17,Direct Indictment,POSS METH,18.2-250,Felony,5,Penitentiary,1095.0,1045.0,Supervised,730.0,Release,NaN,111
223972,2019-02-27,Sentenced,Sentencing,2018-09-17,Direct Indictment,POSS METH,18.2-250,Felony,5,Penitentiary,545.0,455.0,Supervised,545.0,Release,NaN,110
224005,2019-02-22,Sentenced,Plea,2018-11-19,Direct Indictment,POSS METH,18.2-250,Felony,5,Penitentiary,545.0,545.0,Supervised,730.0,Release,NaN,110
224022,2019-02-20,Sentenced,Sentencing,2018-01-18,Indictment,POSS METHAMPHETAMINE,18.2-250,Felony,5,Penitentiary,1825.0,1775.0,Supervised,730.0,Release,NaN,110


In [9]:
# SCH 1/2 transitions

sch1_2_transitions = sch1_2_offenses[['time_step', 'SentenceTime', 'SentenceSuspended', 'ProbationTime']]
sch1_2_transitions = sch1_2_transitions[sch1_2_transitions.time_step <= 120].fillna(0)
sch1_2_transitions.SentenceTime = sch1_2_transitions.SentenceTime.apply(int)
sch1_2_transitions.SentenceSuspended = sch1_2_transitions.SentenceSuspended.apply(int)
sch1_2_transitions['compartment_duration'] = sch1_2_transitions.SentenceTime - sch1_2_transitions.SentenceSuspended
sch1_2_transitions.loc[sch1_2_transitions['compartment_duration'] > MAX_SENTENCE, 'compartment_duration'] = MAX_SENTENCE
sch1_2_transitions['compartment'] = 'incarceration'
sch1_2_transitions['crime_type'] = 'SCH1/2 possession'
sch1_2_transitions['outflow_to'] = 'release'
sch1_2_transitions['total_population'] = 1


diversion_fractions['SCH1/2 possession'] = len(sch1_2_transitions[sch1_2_transitions.compartment_duration == 0]) / len(sch1_2_transitions)

sch1_2_transitions = sch1_2_transitions[sch1_2_transitions.compartment_duration > 0].drop(['SentenceSuspended', 'ProbationTime', 'SentenceTime'], axis=1)
sch1_2_transitions.compartment_duration.describe()

#leave transitions with time_step for creating outflows table, then gets taken out later


count    23318.000000
mean       220.446608
std        346.293575
min          1.000000
25%         65.000000
50%         95.000000
75%        245.000000
max      14600.000000
Name: compartment_duration, dtype: float64

In [10]:
#sch 1/2 outflows_data

sch1_2_outflows = sch1_2_transitions.groupby('time_step').count()[['total_population']].reset_index()
sch1_2_outflows['compartment'] = 'pretrial'
sch1_2_outflows['outflow_to'] = 'incarceration'
sch1_2_outflows['crime_type'] = 'SCH1/2 possession'

sch1_2_transitions = sch1_2_transitions.drop('time_step', axis=1)
sch1_2_outflows.head()

,time_step,total_population,compartment,outflow_to,crime_type
0,1,104,pretrial,incarceration,SCH1/2 possession
1,2,109,pretrial,incarceration,SCH1/2 possession
2,3,143,pretrial,incarceration,SCH1/2 possession
3,4,99,pretrial,incarceration,SCH1/2 possession
4,5,105,pretrial,incarceration,SCH1/2 possession


In [11]:
M1_sentences = drug_crimes[(drug_crimes.ChargeType == 'Misdemeanor') & (drug_crimes.Class == '1')].copy()
M1_sentences['time_step'] = M1_sentences.HearingDate.apply(lambda x: 12 * (int(x[:4]) - 2010) + int(x[5:7]))
M1_sentences = M1_sentences[M1_sentences.time_step <= 120].fillna(0)
M1_sentences.SentenceTime = M1_sentences.SentenceTime.apply(int)
M1_sentences.SentenceSuspended = M1_sentences.SentenceSuspended.apply(int)
M1_sentences['compartment_duration'] = M1_sentences.SentenceTime - M1_sentences.SentenceSuspended
M1_sentences.loc[M1_sentences['compartment_duration'] > MAX_SENTENCE, 'compartment_duration'] = MAX_SENTENCE

diversion_fractions['M1'] = len(M1_sentences[M1_sentences.compartment_duration == 0]) / len(M1_sentences)

M1_sentences = M1_sentences[M1_sentences.compartment_duration > 0]
M1_sentences.compartment_duration.describe()

count    2300.000000
mean      106.905217
std       257.691896
min         1.000000
25%        29.500000
50%        35.000000
75%       125.000000
max      4380.000000
Name: compartment_duration, dtype: float64

In [12]:
# SCH_4

sch4_offenses = drug_crimes[(drug_crimes.CodeSection.str.contains('18.2-250').apply(bool)) 
                              & (drug_crimes.ChargeType == 'Misdemeanor')
                              & (drug_crimes.Class == '2')]
sch4_offenses

,HearingDate,HearingResult,HearingType,Filed,Commencedby,Charge,CodeSection,ChargeType,Class,JailPenitentiary,SentenceTime,SentenceSuspended,ProbationType,ProbationTime,ProbationStarts,AppealedDate,time_step
4680,2010-05-17,Granted,Restricted O.L.,2009-03-06,General District Court Appeal,POSSESS SCH IV,18.2-250,Misdemeanor,2,Jail,30.0,30.0,Unsupervised,730.0,NaN,NaN,5
12775,2010-11-22,Sent,Plea,2010-10-21,Direct Indictment,DRUGS: POSSESS SCH IV,18.2-250,Misdemeanor,2,Jail,180.0,180.0,Supervised,1095.0,Release,NaN,11
16518,2010-12-02,Sent,Trial,2010-09-22,General District Court Appeal,"DRUGS: POSSESS MARIJUANA, 1ST",18.2-250.1,Misdemeanor,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
18892,2010-09-09,Sent,Plea,2010-05-05,General District Court Appeal,"DRUGS: POSSESS MARIJUANA, 1ST",18.2-250.1,Misdemeanor,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
23065,2010-04-05,Sent,Plea,2009-09-30,General District Court Appeal,POSSESSION OF MARIJUANA,18.2-250.1,Misdemeanor,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215085,2019-02-20,Sentenced,Pre-Sentence Report,2018-06-11,Ancillary Misdemeanor,POSS SCH IV DRUGS,18.2-250,Misdemeanor,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110
215536,2019-03-07,Sentenced,Pre-Sentence Report,2018-10-09,Ancillary Misdemeanor,POSSESS CLONAZEPAM,18.2-250,Misdemeanor,2,Jail,365.0,365.0,Indefinite Supervision,NaN,NaN,NaN,111
219644,2019-05-03,Sentenced,Sentencing,2018-11-01,Indictment,DRUGS: POSSESS SCH IV,18.2-250,Misdemeanor,2,Penitentiary,180.0,180.0,Supervised,730.0,Release,NaN,113
220678,2019-01-16,Sentenced,Trial,2019-01-02,Ancillary Misdemeanor,POSS SCH IV DRUG,18.2-250,Misdemeanor,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109


In [13]:
# SCH4 transitions

SCH4_transitions = sch4_offenses[['time_step', 'SentenceTime', 'SentenceSuspended', 'ProbationTime']]
SCH4_transitions = SCH4_transitions[SCH4_transitions.time_step <= 120].fillna(0)
SCH4_transitions.SentenceTime = SCH4_transitions.SentenceTime.apply(int)
SCH4_transitions.SentenceSuspended = SCH4_transitions.SentenceSuspended.apply(int)
SCH4_transitions['compartment_duration'] = SCH4_transitions.SentenceTime - SCH4_transitions.SentenceSuspended
SCH4_transitions.loc[SCH4_transitions['compartment_duration'] > MAX_SENTENCE, 'compartment_duration'] = MAX_SENTENCE
SCH4_transitions['compartment'] = 'incarceration'
SCH4_transitions['crime_type'] = 'SCH4 possession'
SCH4_transitions['outflow_to'] = 'release'
SCH4_transitions['total_population'] = 1


diversion_fractions['SCH4 possession'] = len(SCH4_transitions[SCH4_transitions.compartment_duration == 0]) / len(SCH4_transitions)

SCH4_transitions = SCH4_transitions[SCH4_transitions.compartment_duration > 0].drop(['SentenceSuspended', 'ProbationTime', 'SentenceTime'], axis=1)
SCH4_transitions.compartment_duration.describe()

#leave transitions with time_step for creating outflows table, then gets taken out later


count    161.000000
mean      90.037267
std       90.679097
min        1.000000
25%       30.000000
50%       35.000000
75%      180.000000
max      365.000000
Name: compartment_duration, dtype: float64

In [14]:
#sch 4 outflows_data

SCH4_outflows = SCH4_transitions.groupby('time_step').count()[['total_population']].reset_index()
SCH4_outflows['compartment'] = 'pretrial'
SCH4_outflows['outflow_to'] = 'incarceration'
SCH4_outflows['crime_type'] = 'SCH4 possession'

SCH4_transitions = SCH4_transitions.drop('time_step', axis=1)
SCH4_outflows.head()

,time_step,total_population,compartment,outflow_to,crime_type
0,3,1,pretrial,incarceration,SCH4 possession
1,5,1,pretrial,incarceration,SCH4 possession
2,7,3,pretrial,incarceration,SCH4 possession
3,8,1,pretrial,incarceration,SCH4 possession
4,11,1,pretrial,incarceration,SCH4 possession


In [15]:
M4_sentences = drug_crimes[(drug_crimes.ChargeType == 'Misdemeanor') & (drug_crimes.Class == '4')].copy()
M4_sentences['time_step'] = M4_sentences.HearingDate.apply(lambda x: 12 * (int(x[:4]) - 2010) + int(x[5:7]))
M4_sentences = M4_sentences[M4_sentences.time_step <= 120].fillna(0)
M4_sentences.SentenceTime = M4_sentences.SentenceTime.apply(int)
M4_sentences.SentenceSuspended = M4_sentences.SentenceSuspended.apply(int)
M4_sentences['compartment_duration'] = M4_sentences.SentenceTime - M4_sentences.SentenceSuspended
M4_sentences.loc[M4_sentences['compartment_duration'] > MAX_SENTENCE, 'compartment_duration'] = MAX_SENTENCE

diversion_fractions['M4'] = len(M4_sentences[M4_sentences.compartment_duration == 0]) / len(M4_sentences)

M4_sentences = M4_sentences[M4_sentences.compartment_duration > 0]
M4_sentences.compartment_duration.describe()

count      5.0
mean      90.0
std       60.0
min       30.0
25%       60.0
50%       60.0
75%      120.0
max      180.0
Name: compartment_duration, dtype: float64

# disposition policy

In [28]:
affected_crimes = ['18.2-248', '18.2-250', '18.2-250.1', '18.2-250.2', '18.2-250.3', '18.2-255']
affected_fraction = 1  - len(drug_crimes[drug_crimes.CodeSection.isin(affected_crimes)]) / len(drug_crimes)
affected_fraction


0.3645923899145911

In [17]:
# other

other_offenses = drug_crimes.drop(sch4_offenses.index.union(sch1_2_offenses.index))
other_offenses

,HearingDate,HearingResult,HearingType,Filed,Commencedby,Charge,CodeSection,ChargeType,Class,JailPenitentiary,SentenceTime,SentenceSuspended,ProbationType,ProbationTime,ProbationStarts,AppealedDate,time_step
40,2010-12-02,Sent,Plea,2010-08-10,Indictment,"SALE, DISTRIBUTE MARIJUANA",18.2-248.1,Felony,5,Jail,30.0,26.0,NaN,NaN,NaN,NaN,12
41,2010-12-02,Sent,Plea,2010-09-20,Indictment,UNLAWFULLY POSSESS ALCOHOL,4.1-305(A),Misdemeanor,NaN,Jail,90.0,90.0,NaN,NaN,NaN,NaN,12
109,2010-10-21,Sent,Trial,2010-04-02,Indictment,POSSESS COCAINE,18.2-250,Felony,NaN,Penitentiary,1095.0,1095.0,NaN,NaN,NaN,NaN,10
110,2010-10-21,Sent,Trial,2010-04-02,Indictment,POSSESS MARIJUANA,18.2-250.1,Misdemeanor,NaN,Jail,30.0,30.0,NaN,NaN,NaN,NaN,10
121,2010-10-14,Sent,Pre-Sentence Report,2009-09-10,Indictment,POSS W/INT MANUF/ETC SCH I/II,18.2-248,Felony,NaN,Penitentiary,1825.0,1640.0,Indefinite Supervision,36135.0,Release,NaN,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224197,2019-01-09,Sentenced,Plea,2018-07-18,Indictment,POSS F/A VIO FELON,18.2-308.2,Felony,6,Penitentiary,730.0,730.0,Supervised,180.0,Sentencing,NaN,109
224220,2019-01-09,Sentenced,Sentencing,2018-03-05,Indictment,POSS METH W/I DIST >10G,18.2-248,Felony,NaN,Penitentiary,3650.0,1825.0,Indefinite Supervision,36135.0,Release,NaN,109
224227,2019-01-09,Sentenced,Pre-Sentence Report,2018-06-20,Indictment,POSSESS AMMO FELON,18.2-308.2,Felony,6,Penitentiary,1825.0,1460.0,Supervised,730.0,Release,NaN,109
224228,2019-01-09,Sentenced,Pre-Sentence Report,2018-06-20,Ancillary Misdemeanor,POSSESS MARIJUANA,18.2-250.1,Misdemeanor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109


In [18]:
# other transitions

other_transitions = other_offenses[['time_step', 'SentenceTime', 'SentenceSuspended', 'ProbationTime']]
other_transitions = other_transitions[other_transitions.time_step <= 120].fillna(0)
other_transitions.SentenceTime = other_transitions.SentenceTime.apply(int)
other_transitions.SentenceSuspended = other_transitions.SentenceSuspended.apply(int)
other_transitions['compartment_duration'] = other_transitions.SentenceTime - other_transitions.SentenceSuspended
other_transitions.loc[other_transitions['compartment_duration'] > MAX_SENTENCE, 'compartment_duration'] = MAX_SENTENCE
other_transitions['compartment'] = 'incarceration'
other_transitions['crime_type'] = 'other drug crime'
other_transitions['outflow_to'] = 'release'
other_transitions['total_population'] = 1


diversion_fractions['other drug crime'] = len(other_transitions[other_transitions.compartment_duration == 0]) / len(other_transitions)

other_transitions = other_transitions[other_transitions.compartment_duration > 0].drop(['SentenceSuspended', 'ProbationTime', 'SentenceTime'], axis=1)
other_transitions.compartment_duration.describe()

#leave transitions with time_step for creating outflows table, then gets taken out later


count    40703.000000
mean       534.349385
std        686.264868
min          1.000000
25%         95.000000
50%        365.000000
75%        730.000000
max      14600.000000
Name: compartment_duration, dtype: float64

In [19]:
#other outflows_data

other_outflows = other_transitions.groupby('time_step').count()[['total_population']].reset_index()
other_outflows['compartment'] = 'pretrial'
other_outflows['outflow_to'] = 'incarceration'
other_outflows['crime_type'] = 'other drug crime'

other_transitions = other_transitions.drop('time_step', axis=1)
other_outflows.head()

,time_step,total_population,compartment,outflow_to,crime_type
0,1,323,pretrial,incarceration,other drug crime
1,2,273,pretrial,incarceration,other drug crime
2,3,364,pretrial,incarceration,other drug crime
3,4,293,pretrial,incarceration,other drug crime
4,5,281,pretrial,incarceration,other drug crime


In [20]:
SCH4_outflows

,time_step,total_population,compartment,outflow_to,crime_type
0,3,1,pretrial,incarceration,SCH4 possession
1,5,1,pretrial,incarceration,SCH4 possession
2,7,3,pretrial,incarceration,SCH4 possession
3,8,1,pretrial,incarceration,SCH4 possession
4,11,1,pretrial,incarceration,SCH4 possession
5,12,1,pretrial,incarceration,SCH4 possession
6,13,1,pretrial,incarceration,SCH4 possession
7,14,1,pretrial,incarceration,SCH4 possession
8,15,1,pretrial,incarceration,SCH4 possession
9,17,1,pretrial,incarceration,SCH4 possession


In [21]:
# fill sparse outflows

missing_ts = set(range(1,121)).difference(SCH4_outflows.time_step)
missing_ts_data = pd.DataFrame({'time_step': list(missing_ts)})
missing_ts_data['total_population'] = 0 
missing_ts_data['compartment'] = 'pretrial'
missing_ts_data['outflow_to'] = 'incarceration'
missing_ts_data['crime_type'] = 'SCH4 possession'
SCH4_outflows = SCH4_outflows.append(missing_ts_data)


missing_ts_sch_1_2 = set(range(1,121)).difference(sch1_2_outflows.time_step)
missing_ts_sch_1_2_data = pd.DataFrame({'time_step': list(missing_ts_sch_1_2)})
missing_ts_sch_1_2_data['total_population'] = 0 
missing_ts_sch_1_2_data['compartment'] = 'pretrial'
missing_ts_sch_1_2_data['outflow_to'] = 'incarceration'
missing_ts_sch_1_2_data['crime_type'] = 'SCH1/2 possession'
sch1_2_outflows = sch1_2_outflows.append(missing_ts_data)


missing_ts_other = set(range(1,121)).difference(other_outflows.time_step)
missing_ts_other_data = pd.DataFrame({'time_step': list(missing_ts_other)})
missing_ts_other_data['total_population'] = 0 
missing_ts_other_data['compartment'] = 'pretrial'
missing_ts_other_data['outflow_to'] = 'incarceration'
missing_ts_other_data['crime_type'] = 'other drug crime'
other_outflows = other_outflows.append(missing_ts_other_data)

In [22]:
# probation transitions

temp_probation_transitions = drug_crimes[['time_step', 'ProbationTime', 'ProbationType']]
temp_probation_transitions = temp_probation_transitions[temp_probation_transitions.time_step <= 120].fillna(0)
temp_probation_transitions.ProbationTime = temp_probation_transitions.ProbationTime.apply(int)
temp_probation_transitions.loc[temp_probation_transitions.ProbationType == 'Indefinite Supervision', 'ProbationTime'] = MAX_SENTENCE
temp_probation_transitions = temp_probation_transitions.drop(['ProbationType', 'time_step'], axis=1)
temp_probation_transitions = temp_probation_transitions.rename({'ProbationTime': 'compartment_duration'}, axis=1)
temp_probation_transitions.loc[temp_probation_transitions['compartment_duration'] > MAX_SENTENCE, 'compartment_duration'] = MAX_SENTENCE
temp_probation_transitions = temp_probation_transitions[temp_probation_transitions.compartment_duration > 0]
temp_probation_transitions['compartment'] = 'probation'
temp_probation_transitions['outflow_to'] = 'release'
temp_probation_transitions['total_population'] = 1


probation_transitions = pd.DataFrame()
for crime_type in ['SCH1/2 possession', 'SCH4 possession', 'other drug crime']:
    crime_transitions = temp_probation_transitions.copy()
    crime_transitions['crime_type'] = crime_type
    probation_transitions = pd.concat([probation_transitions, crime_transitions])

temp_probation_transitions.compartment_duration.describe()


count    113739.000000
mean       4115.042817
std        5569.584549
min          12.000000
25%         730.000000
50%        1095.000000
75%        3650.000000
max       14600.000000
Name: compartment_duration, dtype: float64

In [23]:
diversion_fractions

{'SCH1/2 possession': 0.5236071669356651,
 'M1': 0.7142502174183128,
 'SCH4 possession': 0.8081048867699643,
 'M4': 0.9545454545454546,
 'other drug crime': 0.5123108443262284}

In [24]:
# add diversion outflows

outflows_data = pd.DataFrame()
for outflow, tag in [(sch1_2_outflows, 'SCH1/2 possession'), (SCH4_outflows, 'SCH4 possession'), (other_outflows, 'other drug crime')]:
    outflow_probation = outflow.copy()
    outflow_probation.outflow_to = 'probation'
    outflow.total_population *= (1 - diversion_fractions[tag]) / diversion_fractions[tag]
    outflows_data = pd.concat([outflows_data, outflow, outflow_probation])
    



In [25]:
# concat transitions data
release_transitions = pd.DataFrame({'compartment_duration': [MAX_SENTENCE] * 3, 
                                    'compartment': ['release'] * 3, 
                                    'crime_type': outflows_data.crime_type.unique(),
                                    'outflow_to': ['release'] * 3,
                                    'total_population': [1 - 0.217] * 3})
transitions_data = pd.concat([sch1_2_transitions, SCH4_transitions, other_transitions, probation_transitions, release_transitions])
transitions_data.compartment_duration *= 12 / 365

recidivism_transitions = pd.DataFrame({'compartment_duration': range(1,37), 
                                    'outflow_to': ['incarceration'] * 36,
                                    'total_population': [0.217  / 36] * 36})


for crime_type in transitions_data.crime_type.unique():
    crime_recidivism = recidivism_transitions.copy()
    crime_recidivism['compartment'] = 'release'
    crime_recidivism['crime_type'] = crime_type
    transitions_data = transitions_data.append(crime_recidivism)
    
    crime_probation_recidivism = recidivism_transitions.copy()
    crime_probation_recidivism['compartment'] = 'probation'
    crime_probation_recidivism['crime_type'] = crime_type
    transitions_data.loc[(transitions_data.crime_type == crime_type) & 
                         (transitions_data.compartment == 'probation'), 'total_population'] = \
        (1 - 0.217) / len(transitions_data.loc[(transitions_data.crime_type == crime_type) & 
                                               (transitions_data.compartment == 'probation')])
    
    transitions_data = transitions_data.append(crime_probation_recidivism)

    


In [26]:
transitions_data.groupby(['crime_type', 'compartment', 'outflow_to']).sum()

compartment_duration  \
crime_type        compartment   outflow_to                            
SCH1/2 possession incarceration release                1.689986e+05   
                  probation     incarceration          6.660000e+02   
                                release                1.538764e+07   
                  release       incarceration          6.660000e+02   
                                release                4.800000e+02   
SCH4 possession   incarceration release                4.765808e+02   
                  probation     incarceration          6.660000e+02   
                                release                1.538764e+07   
                  release       incarceration          6.660000e+02   
                                release                4.800000e+02   
other drug crime  incarceration release                7.150561e+05   
                  probation     incarceration          6.660000e+02   
                                release                1.538764e+07   
                  release       incarceration          6.660000e+02   
                                release                4.800000e+02   

                                               total_population  
crime_type        compartment   outflow_to                       
SCH1/2 possession incarceration release               23318.000  
                  probation     incarceration             0.217  
                                release                   0.783  
                  release       incarceration             0.217  
                                release                   0.783  
SCH4 possession   incarceration release                 161.000  
                  probation     incarceration             0.217  
                                release                   0.783  
                  release       incarceration             0.217  
                                release                   0.783  
other drug crime  incarceration release               40703.000  
                  probation     incarceration             0.217  
                                release                   0.783  
                  release       incarceration             0.217  
                                release                   0.783

In [27]:
# from recidiviz.calculator.modeling.population_projection.spark_bq_utils import upload_spark_model_inputs
# upload_spark_model_inputs('recidiviz-staging', 'VA_HB2303', outflows_data, transitions_data, pd.DataFrame())